In [1]:
import pandas as pd

In [2]:
import BRADA as brada
print(brada.__dist_name__)
print('Version:', brada.__version__)

BRADA - BR Distribuidora Advanced Analytics Framework
Version: 0.0.1


In [3]:
pequim = pd.read_csv('datasets/BeijingPM20100101_20151231.csv')
pequim.dropna(inplace=True)

pequim['date'] = pequim['year'].astype(str) + '-' + pequim['month'].astype(str) + '-' + pequim['day'].astype(str)
pequim.drop_duplicates(subset='date', inplace=True)
pequim['date'] = pd.to_datetime(pequim['date'])
pequim.sort_values('date', inplace=True)
pequim.set_index('date', inplace=True)
pequim.index = pd.DatetimeIndex(pequim.index).to_period('D')

pequim.columns

Index(['No', 'year', 'month', 'day', 'hour', 'season', 'PM_Dongsi',
       'PM_Dongsihuan', 'PM_Nongzhanguan', 'PM_US Post', 'DEWP', 'HUMI',
       'PRES', 'TEMP', 'cbwd', 'Iws', 'precipitation', 'Iprec'],
      dtype='object')

In [4]:
black_friday = pd.read_csv('datasets/BlackFriday.csv')
black_friday.columns                           

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [5]:
fb = pd.read_csv('datasets/FB.csv')
fb.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [6]:
fb['Date'] = pd.to_datetime(fb['Date'])
fb = fb.set_index('Date')

In [2]:
from lime import lime_tabular

In [ ]:
lime_tabular.LimeTabularExplainer()

In [3]:
2667.03-3251.62

-584.5899999999997

In [4]:
a = set()

In [5]:
a.add('a')

In [6]:
a

{'a'}

In [14]:
import pipreqs
import os

In [18]:
brada_path = os.getcwd() + '\BRADA'

In [19]:
brada_path

'C:\\Users\\8qyk\\OneDrive - BR\\BRADA\\BRADA'

In [27]:
os.system('pipreqs --encoding=utf8')

0

'C:\\Users\\8qyk\\OneDrive - BR\\BRADA'